# Programming for Chemists: Physical Chemistry in Python

**Importance for scientists:**
* The purpose of this session is to learn a variety of tools applicable in many different scientific applications on the job, including:
    * Symbolic computation.
    * Finding maxima of functions.
    * Automatically varying fitting parameters to minimize or maximize certain values.

* This hands on session will explore the hydrogen atom in intricate detail, adding more tools to your programming toolbox. 
* First we discuss a very useful Python library called [SymPy](https://www.sympy.org/en/index.html). SymPy is a Python library for symbolic mathematics; aiming to become a full-featured **computer algebra system (CAS)**. 

## Computer Algebra in SymPy 

* Computer algebra refers to software packages that are capable of doing **symbolic mathematical computations** such as:

\\[
    (x + 1)^2 \hspace{0.5cm} \rightarrow \hspace{0.5cm} x^2 + 2x + 1,
\\]
* where the word "algebraic" refers to operations with symbolic objects originating from algebra. 
* Two well known software packages allowing for this functionality are *Maple* and *Mathematica*, but Python has a library called **SymPy** which provides us the ability to do symbolic computation in Python. 
* Scientific computing is usually based on **numerical** computation using **approximate floating point numbers**, while symbolic computation emphasises **exact** computation with expressions containing variables that have no given value and are manipulated as symbols. 


| Symbolic Pros                        | Symbolic Cons                                                      |
|:-------------------------------------|:-------------------------------------------------------------------|
| Exact Arithmetic                     | **Much slower** than numerical computation                         |
| Non-linear expressions with symbols  | Poor hardware support                                              |
| Inequalities, differential equations | Difficult to parallelize                                           |
| Results can be formulae              | Numerical problems better handled with "orthodox" computer methods |
| It "knows" how to integrate          | Number of expressions can grow exponentially                       |

### Example: Analytical vs. Numerical Integration

Consider the [known integral](https://functions.wolfram.com/Constants/Pi/07/01/01/0002/)

\\[
    4\int\limits_{0}^{1}\sqrt{1-t^2}\text{d}t = \pi.
\\]

* We pass this integral to SymPy and it produces \\(\pi\\) **exactly**, not a floating point representation (we will discuss SymPy terminology throughout this tutorial):

In [ ]:
import sympy as sy

# define t as symbol not requiring an explicit value
t = sy.symbols('t')

# define the integrand
integrand = sy.sqrt(1 - t**2)

# calculate the integral
4*sy.integrate(integrand, (t, 0, 1))

* Native programming languages; Python, C++, Fortran, Java etc... **cannot do this** without the help of customised libraries, but can solve the integral **approximately** using **numerical** methods. 
* I programmed the following animation using matplotlib simulating a numerical technique called a **trapezoid Riemann sum** of the same integral above; approximating the area under the function using an increasing number of trapezoids:

<center><img src="https://raw.githubusercontent.com/adambaskerville/ProgrammingForChemists/master/images/riemann_sum.gif" width="900" height="900" /></center>

## Solving the Hydrogen Atom 

* In this section we will conduct a detailed investigation on how to solve the hydrogen atom; the building block of a lot of computational and quantum chemical methods. 
* It can be described using the **time independent, non-relativistic Schr&ouml;dinger equation**

\\[
 \hat{H}\psi = E\psi,
 \tag{1}
\\]

* where: 
    * \\(\hat{H} \\) is the Hamiltonian, the total energy operator.
    * \\(\psi\\) is the wavefunction, containing all the key physical behaviour of the system.
    * \\(E\\) is the energy eigenvalue. 

* This is the textbook system studied in quantum mechanics as it is one of the most simple, consisting of a single proton and electron. 
    * The radial part of the hydrogen atom is **exactly solvable** using a series solution method but lets pretend we are unaware of this and explore how a computer can solve it. 
    * The aim will be to get the computer to do *everything*, requiring no preliminary mathematics by us humans. 
    * We are going to use a very powerful mathematical technique called the **variational method**.

### Variational Method 

\\[
\frac{\int\psi^* \hat{H} \psi \text{d}\nu}{\int \psi^* \psi \text{d}\nu} \ge E_g.
\tag{2}
\\]

* If we do not know the exact wavefunction our best bet is to have an educated guess using some physical intuition; unlikely to be the exact wavefunction, but by labelling it as \\(\psi_1\\) it will calculate a value of the energy, \\(E_1\\). 

* We can now guess a second wavefunction, \\(\psi_2\\), resulting in an energy value \\(E_2\\) and if desired we could guess even more wavefunctions.
* The variational principle tells us that if \\(E_g\\) is the exact ground state energy then both \\(E_1\\) and \\(E_2\\) will always be greater than \\(E_g\\) unless we somehow guess the exact wavefunction; which would then be equal to \\(E_g\\).
* Due to the inability of solving many-body problems exactly, computational and quantum chemistry is built upon approximate methods and a number of methods utilize the variational principle, including **Hartree Fock theory**. 
    * These work by making repeated iterative changes to the trial wavefunctions or density matrices, calculating the energy as they go until the lowest value has been found; safe in the knowledge it can not drop below the exact ground state solution. 

### Gaussian Wavefunction

* For our first trial wavefunction, \\(\psi_1\\), we will use a single [Gaussian function](https://mathworld.wolfram.com/GaussianFunction.html) 

\\[
\psi_1 = e^{-cr^2},
\\]

* where \\(r\\) is the nucleus-electron distance and \\(c\\) is a **variational parameter** which our program will change throughout the calculation in order to find the lowest energy, \\(E_g\\). 

* <font color='red'>**Exercise:** Plot the Gaussian function when $c=1$ using matplotlib:</font>

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

r = np.arange(-2, 2, 0.1)

* The Hamiltonian for the hydrogen atom in atomic units is given by\\(^{[1]}\\)

\\[
\hat{H} = -\overbrace{\frac{1}{2}\nabla^2}^{K.E} - \overbrace{\frac{1}{r}}^{P.E}.
\tag{5}
\\]

* The Laplacian operator for the **spherically symmetric 1s state** of the hydrogen atom is

\\[
    \nabla^2 = \frac{\partial^2}{\partial r^2} + \frac{2}{r}\frac{\partial}{\partial r}.
\\]

* The first thing we do is import NumPy, SciPy, SymPy and a useful module within the SciPy library called `Optimize` containing a lot of functionality relevant for non-linear optimization problems. We want to import the `minimize` function which we do using

    `from scipy.optimize import minimize`

* As we are going to solve this symbolically using computer algebra we need to tell Python to treat the coordinate `r` as a symbol not requiring an explicit value. We do this using `sy.symbols('r')` from SymPy:

    `r = sy.symbols('r')`

* which tells the program to treat `r` as a generic variable not needing an explicit value.

* <font color='red'>**Exercise:** Import numpy, scipy, sympy and the minimize function from scipy:</font>
* <font color='red'>**Exercise:** Tell Python to treat r as a symbol not a numerical value using sympy:</font>

**Define wavefunction:**

* The next step is to tell the program what our Gaussian wavefunction looks like:

    `psi_1 = sy.exp(-c*r**2)`

* <font color='red'>**Exercise:** program this wavefunction into the following code box:</font>

* where we call the **SymPy** implementation of the exponential function. 

**Define Hamiltonian:**

* We next define the Hamiltonian operator and get SymPy to evaluate the differential operators for us, **symbolically!** 
    * Remember an operator, our Hamiltonian, only makes sense if it **is acting on something** so we will act it on the wavefunction on its right hand side of equation (2):

        `hamiltonian = (-1/2)*(sy.diff(psi_1, r, r) + (2/r)*sy.diff(psi_1, r)) - (1/r)*psi_1`

* There is a lot to unpack here:

    1. `sy.diff` calls the differential operator from SymPy. The first differential in our \\(\nabla^2\\) operator is a second order derivative with respect to \\(r\\) which can be programmed in two equivalent ways:

        `sy.diff(psi_1, r, r)` or `sy.diff(psi_1, r, 2)`

    2. the next derivative is first order:

        `sy.diff(psi_1, r)`

    3. we also need to multiply the potential energy term at the end by `psi_1` as it is part of the Hamiltonian operator.
    
* <font color='red'>**Exercise:** program the Hamiltonian into the following code box and print it. What has sympy done?</font>

**Volume of integration:**

* Quantum mechanics tells us the wavefunction gives the probability amplitude for finding the electron in a particular region of space d\\(\nu\\)
    * For the spherically symmetric 1s state of the hydrogen atom this region of space is a sphere surrounding the proton. 
    * The surface area of a sphere is \\(4\pi r^2\\) which we will program as our volume element, `dv`, remember to use `sy.pi`.
    
* <font color='red'>**Exercise:** program the volume element into the following code box:</font>

**Integration:**

* We have now defined our wavefunction, Hamiltonian operator and volume element, so we can now conduct the integration. 
* We will implement two expressions, one for the numerator (`num`) of equation (2) and one for the denominator (`den`):

    `num = sy.N(sy.integrate(psi_1*hamiltonian*dv, (r, 0, sy.oo)))`

* Lets break this down:

    1. To symbolically integrate in SymPy we call:

        `sy.integrate(integrand, (variable to integrate over, start, end))` 

    2. We want to integrate over \\(r\\) from 0 to \\(\infty\\) as the theory of quantum mechanics tells us probability densities are asymptotic at long-range; but the probability is **effectively zero** past a radius of several Bohr radii.

        `(r, 0, sy.oo)`

    infinity is represented as `oo` in Sympy. 

    3. As SymPy evaluates it symbolically, it will leave it in a general form (print out the result to check for yourself!) and we want a numeric answer so call `sy.N` which numerically evaluates the expressions for us.
    
* <font color='red'>**Exercise:** Program the denominator in the following code box and divide with the numerator to get the energy expression:</font>. 

* **We are finished!** 
*  <font color='red'>Move your code segments into the following function and run the cell</font>. 

In [ ]:
def min_energy(c):
    '''
    This function minimises the energy with respect to a single variational parameter c
    '''
    # The scipy minimize function passes the parameters in an array. 
    # To use it within our mathematical formulae we need to extract it as a separate number, done using the index [0]
    c = c[0]
    
    # define the Gaussian wavefunction

    # define the Hamiltonian

    # define the volume of integration

    # define the numerator of the variational principle

    # define the denominator of the variational principle

    # calculate the energy
    E = num/den
    
    # return E to the minimize function
    return E

* We are going to minimize this function using the **minimize** function from **scipy**:
    1. Set an initial value for c.
    2. Call the minimize function using the syntax:
    
        `minimize(term to minimize, variables to minimize, tol=convergence tolerance, bounds=boundaries for your parameters)`
    3. Minimize will pass the variables inside a numpy array to our function which we need to extract when inside the function.
    4. We apply boundaries in this example as otherwise scipy will make c negative which will seize up the function whilst sympy tries to solve the difficult integration.
* <font color='red'>**Exercise:** Add the variable to be varied into the following code block and run:</font>

In [ ]:
# set an initial value for the variational parameter 'c'
c = 3

bnds = ((0,10),)

# call the minimize function, tell it the parameter to vary, c, and tell it the tolerance for convergence, tol
hydrogen_gs = minimize(min_energy, tol=1e-6, bounds=bnds)

# print the final result to terminal
print(hydrogen_gs)

# extract out the value of the variational parameter and the energy value
print("\nEnergy = {:.10f} hartrees when c = {:.6f}".format(hydrogen_gs.fun, hydrogen_gs.x[0]))

* We have a final energy of \\(E_g = -0.4244131816\\) hartrees when \\(c = 0.28294212\\).
* We know the exact ground state energy for a one-electron system

\\[
E_g = -\frac{Z^2}{2},
\\]

* where \\(Z\\) is the nuclear charge, so for the hydrogen atom \\(E = -0.5\\) hartrees. Using a Gaussian wavefunction produces an **Energy value with a \\(\approx 15\%\\) error!**

### Slater-type orbital Wavefunction

* Let us consider a better approximation for our wavefunction, this time using a **Slater-type orbital**:

\\[
\psi_2 = e^{-cr}.
\\]

 <font color='red'>**Exercise:** Add the slater function to the plot below:</font>

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

# create the figure (fig) and axes (ax) objects
fig, ax = plt.subplots()

# create r range from 0 -> 5 with a step of 0.05
r = np.arange(0, 5, 0.05)   # start, stop, step

# define gaussian function
gaussian = np.exp(-r**2)

# define slater function


# set the range of the x-axis data
ax.set_xlim([0, 5])

# set the range of the y-axis data
ax.set_ylim([0, 1])

# set the axis labels
ax.set_xlabel('r')
ax.set_ylabel(r'$\psi(r)$')

# plot the Gaussian function
ax.plot(r, gaussian, label = 'Gaussian')


# plot the legend
ax.legend()

plt.show()

* These two forms look quite different, and the key reason why the Slater form is more **physically realistic** than the Gaussian form is that it reaches a **cusp** at the origin, \\(r=0\\), which is a rule particles have to obey when they coalesce; [rigorously proven](https://onlinelibrary.wiley.com/doi/abs/10.1002/cpa.3160100201) by Tosio Kato in 1957.

* <font color='red'>**Exercise:** Return to your program, or the complete version at the bottom of the script, and change the Gaussian function for a Slater-type function and see what the new result is.</font>

    * We end up with \\(E_g = -0.5\\) hartrees when \\(c=1\\), matching the exact energy to the specified precision. 
* This also reveals the nature of the exact wavefunction of the hydrogen atom but also any one-electron system

\\[
\psi = e^{-Zr},
\\]

* where \\(Z\\) is the nuclear charge. 

*  <font color='red'>**Exercise:** Complete the for loop below to plot the exact wavefunctions (r against slater) with differing nuclear charges:</font>

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

# set a dark grid style for the plot (looks nice!)
sns.set_style("darkgrid")

# create the figure (fig) and axes (ax) objects
fig, ax = plt.subplots()

# create r range from 0 -> 5 with a step of 0.05
r = np.arange(0, 5, 0.05)   # start, stop, step

# set the range of the x-axis data
ax.set_xlim([0, 5])

# set the range of the y-axis data
ax.set_ylim([0, 1])

# set the axis labels
ax.set_xlabel('r')
ax.set_ylabel(r'$\psi(r)$')

for  in range(1,5):
    # define slater function
    slater = 
    
    # call the plot command
    ax.plot(, slater, label = "Z = {}".format(Z))

    # plot the legend

plt.show()

### Radial Distribution Functions
We now have our optimised Gaussian and Slater type wavefunctions for the hydrogen atom

**Gaussian:**

\\[
\psi_1(r) = e^{-0.28294212 r}
\\]

**Slater:**

\\[
\psi_2(r) = e^{-1r}
\\]

* We can use these wavefunctions to calculate the **radial Distribution Function (RDF)** of the hydrogen atom; i.e. the electron density around the proton

\\[
    \text{RDF} = 4\pi r^2\psi(r)\psi(r)|_{r=r'}
    \tag{7}
\\]

* where we fix the coordinate \\(r\\) to be values in the range \\(0 \le r' \le \infty\\). 
* which we need only calculate **once** using SymPy, and then divide the RDF function by the normalisation value. 
* We are going to make use of the [`lambdify`](https://docs.sympy.org/latest/modules/utilities/lambdify.html) function from SymPy which transforms SymPy expressions to a numpy recognisable format meaning we can calculate numerical values very fast. 

* <font color='red'>**Exercise:** Add the code to calculate and plot the RDF for the slater wavefunction:</font>

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import sympy as sy
from sympy import lambdify

r = sy.symbols('r')

# create the figure (fig) and axes (ax) objects
fig, ax = plt.subplots()

# define gaussian function
gaussian = sy.exp(-r**2)

# define slater function
slater = sy.exp(-r)

# define volume element
dv = 4*sy.pi*r**2

# calculate the inner product <psi|psi> of the gaussian wavefunction
gaussian_norm = sy.N(sy.integrate(gaussian*gaussian*dv, (r, 0, sy.oo)))

# calculate the inner product <psi|psi> of the slater-type orbital wavefunction
slater_norm = sy.N(sy.integrate(slater*slater*dv, (r, 0, sy.oo)))

# calculate the RDF using the Gaussian wavefunction
RDF_gaussian = gaussian*gaussian*dv / gaussian_norm
lam_gaussian = lambdify(r, RDF_gaussian) # use lambdify to then treat as a function

# calculate the RDF using the slater-type orbital wavefunction

# set the range of the x-axis data
ax.set_xlim([0, 5])

# set the range of the y-axis data
ax.set_ylim([0, 1.5])

# set the axis labels
ax.set_xlabel("r'")
ax.set_ylabel(r'$4\pi r^2\psi(r)\psi(r)$') # This notation uses LaTeX math font!

# generate 500 r' values (rp) between 0 and 5
rp_vals = np.linspace(0, 5, 500)  # start, stop, no. points 

# calculate the y-values of the graph using the RDF functions
y_vals_gaussian = lam_gaussian(rp_vals)

# call the plot command
ax.plot(rp_vals, y_vals_gaussian, label = 'RDF Gaussian')

# plot the legend
ax.legend()

plt.show()

* We can check they are normalised correctly by integrating the RDF functions to calculate the area underneath the curves.
* <font color='red'>**Exercise:** Write the code for the slater-type orbital wavefunction:</font>:

In [ ]:
# check normalisation of the RDF using a gaussian wave function
print(sy.N(sy.integrate(RDF_gaussian, (r, 0, sy.oo))))

# check normalisation of the RDF using a gaussian wave function


### Most Probable Value of r

* If we look at the RDF plots above, they are not symmetric about a point like a standard Gaussian function is; meaning the **most probable** and **average** value of \\(r\\) are not equal! 
* The most probable value is given by the position of the maximum of the curve. To find and classify extreme points of a function take the first derivative and set equal to 0 and solve. 
* As we know the function has a maximum by visual inspection we do not need to take the second derivative in order to classify whether it is a maximum, minimum or point of inspection, 

In [ ]:
def find_extremes(func, arg):
    '''
    This function find the extreme points of a sympy function, returning the maximum value
    
    Parameters
    ----------
    func : sympy.core.mul.Mul
           This is the symbolic sympy expression which we want to find the maximum of
        
    arg : sympy.core.symbol.Symbol
          This is the variable to differentiate wirht respect to.
    
    Returns
    -------
    max(extremes) : The maximum value in the extreme points list
    '''
    # calculate first derivative
    dy = sy.diff(func, arg)
    
    # calculate extreme points
    extremes = sy.solve(dy, arg)

    return max(extremes)

print("Most probable r for Gaussian = {:.3f}".format(find_extremes(RDF_gaussian, r)))

print("Most probable r for Slater = {:.3f}".format(find_extremes(RDF_slater, r)))

* The most probable value of \\(r\\) for hydrogen is exactly \\(r = 1\\) a.u. so this shows that the Gaussian wavefunction does not just underestimate the ground state energy, but also the physical properties of the system.

## Review

Today we covered:

* Differences between symbolic and numerical computation.
* How sympy allows for symbolic mathematics to be used in solving a problem.
* How to symbolically differentiate and integrate in sympy.
* How to program a minimisation problem to solve the hydrogen atom.
* How to find maxima of functions.

## Exercise

Calculate the expectation value of the nucleus-electron distance using sympy for both the Gaussian and Slater wavefunctions. Expectation values represent the average value of a given quantity, which in the case of the proton-electron distance, \\(r\\), in the hydrogen atom is

\\[
    \langle \psi(r)| r |\psi(r) \rangle = \frac{\int\limits_{0}^{\infty} \psi(r) r \psi(r) \text{d}\nu}{\int\limits_{0}^{\infty}\psi(r)\psi(r)\text{d}\nu}
    \tag{9}
\\]

**If you get stuck the answer code is at the bottom of the notebook:**

## Complete Hydrogen Atom Program

In [ ]:
import numpy as np
import scipy as sp
import sympy as sy
from scipy.optimize import minimize

r = sy.symbols('r')

def min_energy(c):
    '''
    This function minimises the energy with respect to a single variational parameter c
    
    Parameters:
    -----------
    c : np.ndarray
        This is the current value of the c parameter
    
    Returns:
    --------
    E : sympy.core.numbers.Float
        This number represents the energy value in hartrees

    '''
    # The scipy minimize function passes the parameters in an array. 
    # To use it within our mathematical formulae we need to extract it as a separate number, done using the index [0]
    c = c[0]

    # define the Gaussian wavefunction
    psi = sy.exp(-c*r**2)

    # define the Hamiltonian
    hamiltonian = (-1/2)*(sy.diff(psi, r, r) + (2/r)*sy.diff(psi, r)) - (1/r)*psi

    # define the volume of integration
    dv = 4*sy.pi*r**2

    # define the numerator of the variational principle
    num = sy.N(sy.integrate(psi*hamiltonian*dv, (r, 0, sy.oo)))

    # define the denominator of the variational principle
    den = sy.N(sy.integrate(psi*psi*dv, (r, 0, sy.oo)))

    # calculate the energy
    E = num/den

    # return E to the minimize function
    return E

# set an initial value for the variational parameter 'c'
c = 3
bnds = ((0,10),)

# call the minimize function, tell it the parameter to vary, c, and tell it the tolerance for convergence
hydrogen_gs = minimize(min_energy, c, tol=1e-6, bounds=bnds)

print(hydrogen_gs)

print("\nEnergy = {:.10f} hartrees when c = {:.6f}".format(hydrogen_gs.fun, hydrogen_gs.x[0]))

## Expectation value answer

In [ ]:
# define gaussian function
gaussian = sy.exp(-r**2)

# define gaussian function
slater = sy.exp(-r)

# define the volume of integration
dv = 4*sy.pi*r**2
    
# gaussian expectation value of r
expec_num_gaussian = sy.N(sy.integrate(gaussian*r*gaussian*dv, (r, 0, sy.oo)))
expec_den_gaussian = sy.N(sy.integrate(gaussian*gaussian*dv, (r, 0, sy.oo)))

print("<r> using Gaussian = {:.4f}".format(expec_num_gaussian / expec_den_gaussian))

# slater-type expectation value of r
expec_num_slater = sy.N(sy.integrate(slater*r*slater*dv, (r, 0, sy.oo)))
expec_den_slater = sy.N(sy.integrate(slater*slater*dv, (r, 0, sy.oo)))

print("<r> using Slater = {:.4f}".format(expec_num_slater / expec_den_slater))

## Further Reading

* If you found this example interesting, then I have written similar posts on my T>T blog:
    * A [very comprehensive guide to programming Hartree Fock in Python](https://adambaskerville.github.io/posts/HartreeFockGuide/). 
    * A guide on how to program a gas molecule following [Brownian motion](https://adambaskerville.github.io/posts/BrownianMotion/). 
    * Two guides on how to program a Langton's ant Turing machine; which produces order (and fractals!) from seemingly random movement patterns. Langton's ant is also one of the challenge problems from Project Euler:

    [Langton's ant](https://adambaskerville.github.io/posts/LangtonsAnt/)

    [Extension to Langton's ant to produce fractals](https://adambaskerville.github.io/posts/LangtonsAnt2ElectricBoogaloo/)


# References

[1] Quantum Mechanics, F. Mandl, 1992, Wiley & Sons, page 187.